In [1]:
from copy import deepcopy

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError

In [2]:
CONFIGS = {
    'data_path': '../data/',
    'model_path': '../model/',
    'model_name': 'super_basic',

    'valid_start_index': 1704,
    'test_start_index': 1872,
    
    'batch_size': 64,
    'learning_rate': 1e-4,
    'epochs': 100,
    'es_patience': 10,
    
    'window_size': 7*24,
    'target_length': 3,
}

In [3]:
data_path = '../data/'

train_origin = pd.read_csv(data_path+'train.csv', encoding='cp949')

In [4]:
data = deepcopy(train_origin)

data.columns = [
    'num', 'date_time', 'target', 'temp', 'wind',
    'humid', 'rain', 'sun', 'non_elec_eq', 'sunlight_eq'
]

data = data.loc[data['num'] == 1, ['date_time', 'target']]

print(f'data.shape: {data.shape}')

data.shape: (2040, 2)


In [5]:
def mk_mean_std_dict(data):
    mean_std_dict = {
        col: {
            'mean': data[col].mean(),
            'std': data[col].std()
        } for col in data.columns
    }
    return mean_std_dict

In [6]:
scaling_cols = ['target']

mean_std_dict = mk_mean_std_dict(data[scaling_cols][:CONFIGS['valid_start_index']])
CONFIGS['mean_std_dict'] = mean_std_dict

In [7]:
def standard_scaling(data, mean_std_dict=None):
    if not mean_std_dict:
        mean_std_dict = mk_mean_std_dict(data)
    new_data = data.copy()
    for col in new_data.columns:
        new_data[col] -= mean_std_dict[col]['mean']
        new_data[col] /= mean_std_dict[col]['std']
    return new_data

In [8]:
data[scaling_cols] = standard_scaling(data[scaling_cols])

In [9]:
input_cols = [f't-{i}' for i in range(CONFIGS['window_size'], 0, -1)]
target_cols = [f't+{i}' for i in range(CONFIGS['target_length'])]

CONFIGS['input_cols'] = input_cols
CONFIGS['target_cols'] = target_cols

In [10]:
def mk_time_series(data):
    
    new_data_length = data.shape[0]-CONFIGS['window_size']-CONFIGS['target_length']+1
    new_data_shape = (new_data_length, CONFIGS['window_size']+CONFIGS['target_length'])
    new_data = np.zeros(new_data_shape)

    for i in range(new_data_length):
        new_data[i, :CONFIGS['window_size']] = data['target'][i:i+CONFIGS['window_size']]
        new_data[i, CONFIGS['window_size']:] = \
            data['target'][i+CONFIGS['window_size']:i+CONFIGS['window_size']+CONFIGS['target_length']]

    new_data = pd.DataFrame(new_data)
    new_data.columns = input_cols + target_cols
    
    return new_data

In [11]:
def split_data(data, CONFIGS):
    
    train = data[:CONFIGS['valid_start_index']]
    valid = data[
        CONFIGS['valid_start_index']-CONFIGS['window_size']:CONFIGS['test_start_index']
    ]
    test = data[
        CONFIGS['test_start_index']-CONFIGS['window_size']:
    ]
    
    train, valid, test = \
        mk_time_series(train), mk_time_series(valid), mk_time_series(test)
    
    return train, valid, test

In [12]:
train, valid, test = split_data(data, CONFIGS)

In [13]:
def inversed_rmse(y_true, y_pred, mean, std):
    y_true = (y_true+mean)*std
    y_pred = (y_pred+mean)*std
    mse = tf.reduce_mean((y_true-y_pred)**2)
    return tf.sqrt(mse)

inversed_rmse_metric = lambda y_true, y_pred: inversed_rmse(y_true, y_pred, **CONFIGS['mean_std_dict']['target'])

In [14]:
def set_model(CONFIGS, model_name = None, print_summary=False):
    inputs = Input(batch_shape=(None, CONFIGS['window_size']), name='inputs')
    dense_0 = Dense(64, activation='relu', name='dense_0')(inputs)
    dense_1 = Dense(32, activation='relu', name='dense_1')(dense_0)
    outputs = Dense(CONFIGS['target_length'], name='outputs')(dense_1)
    
    if not model_name:
        model_name = CONFIGS['model_name']
    
    model = Model(
        inputs, outputs,
        name = model_name
    )
    
    optimizer = Adam(learning_rate=CONFIGS['learning_rate'])
    model.compile(
        loss = MeanSquaredError(),
        optimizer = optimizer,
        metrics=[inversed_rmse_metric],
    )
    
    if print_summary:
        model.summary()
    
    return model

In [15]:
model = set_model(CONFIGS, print_summary=True)

Model: "super_basic"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 168)]             0         
_________________________________________________________________
dense_0 (Dense)              (None, 64)                10816     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
outputs (Dense)              (None, 3)                 99        
Total params: 12,995
Trainable params: 12,995
Non-trainable params: 0
_________________________________________________________________


In [16]:
def train_model(model, train, valid, CONFIGS):
    
    X_train, y_train = train[CONFIGS['input_cols']], train[CONFIGS['target_cols']]
    X_valid, y_valid = valid[CONFIGS['input_cols']], valid[CONFIGS['target_cols']]
    
    early_stop = EarlyStopping(
        patience=CONFIGS['es_patience']
    )
    save_best_only = ModelCheckpoint(
        filepath = f'{CONFIGS["model_path"]}{CONFIGS["model_name"]}.h5',
        monitor = 'val_loss',
        save_best_only = True,
        save_weights_only = True
    )
    
    history = model.fit(
        X_train, y_train,
        batch_size = CONFIGS['batch_size'],
        epochs = CONFIGS['epochs'],
        validation_data = (X_valid, y_valid),
        callbacks = [
            early_stop,
            save_best_only,
        ]
    )
    
    return history

In [17]:
history = train_model(model, train, valid, CONFIGS)

Epoch 1/100
24/24 [==============================] - 1s 6ms/step - loss: 0.8748 - <lambda>: 117.3243 - val_loss: 3.1441 - val_<lambda>: 224.4263
Epoch 2/100
24/24 [==============================] - 0s 2ms/step - loss: 0.5661 - <lambda>: 94.2298 - val_loss: 1.9884 - val_<lambda>: 177.2849
Epoch 3/100
24/24 [==============================] - 0s 2ms/step - loss: 0.4123 - <lambda>: 80.7562 - val_loss: 1.4173 - val_<lambda>: 148.4519
Epoch 4/100
24/24 [==============================] - 0s 2ms/step - loss: 0.3322 - <lambda>: 72.3994 - val_loss: 1.0957 - val_<lambda>: 129.4079
Epoch 5/100
24/24 [==============================] - 0s 2ms/step - loss: 0.2839 - <lambda>: 66.9278 - val_loss: 0.8781 - val_<lambda>: 114.8648
Epoch 6/100
24/24 [==============================] - 0s 2ms/step - loss: 0.2504 - <lambda>: 62.7858 - val_loss: 0.7375 - val_<lambda>: 104.5329
Epoch 7/100
24/24 [==============================] - 0s 2ms/step - loss: 0.2263 - <lambda>: 59.6391 - val_loss: 0.6362 - val_<lambda>: 

Epoch 58/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0628 - <lambda>: 30.7586 - val_loss: 0.1504 - val_<lambda>: 47.6734
Epoch 59/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0622 - <lambda>: 30.7842 - val_loss: 0.1463 - val_<lambda>: 46.9352
Epoch 60/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0616 - <lambda>: 30.4558 - val_loss: 0.1481 - val_<lambda>: 47.2193
Epoch 61/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0609 - <lambda>: 30.5113 - val_loss: 0.1451 - val_<lambda>: 46.7269
Epoch 62/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0605 - <lambda>: 30.1405 - val_loss: 0.1486 - val_<lambda>: 47.3940
Epoch 63/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0597 - <lambda>: 29.8268 - val_loss: 0.1453 - val_<lambda>: 46.7920
Epoch 64/100
24/24 [==============================] - 0s 2ms/step - loss: 0.0593 - <lambda>: 29.8462 - val_loss: 0.1432 - val_<lambda>: 

In [18]:
best_model = set_model(CONFIGS, model_name='best_'+CONFIGS['model_name'])
best_model.load_weights(f'{CONFIGS["model_path"]}{CONFIGS["model_name"]}.h5')

In [19]:
X_train, y_train = train[CONFIGS['input_cols']], train[CONFIGS['target_cols']]
X_valid, y_valid = valid[CONFIGS['input_cols']], valid[CONFIGS['target_cols']]
X_test, y_test = test[CONFIGS['input_cols']], test[CONFIGS['target_cols']]

y_train_pred = best_model.predict(X_train)
y_valid_pred = best_model.predict(X_valid)
y_test_pred = best_model.predict(X_test)

In [20]:
train_loss = best_model.evaluate(X_train, y_train, verbose=0)
valid_loss = best_model.evaluate(X_valid, y_valid, verbose=0)
test_loss = best_model.evaluate(X_test, y_test, verbose=0)

print(f'train_loss: {train_loss}')
print(f'valid_loss: {valid_loss}')
print(f'test_loss: {test_loss}')

train_loss: [0.04459547623991966, 24.093278884887695]
valid_loss: [0.13072478771209717, 42.21330642700195]
test_loss: [0.10923188924789429, 40.172481536865234]
